## Imports

In [1]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from tqdm.notebook import tqdm

import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import imblearn
import optuna
import torch
import copy
import json
import os

warnings.filterwarnings("ignore", category=DeprecationWarning) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load CLS-tokens and map 'incomplete-classes' to their respective full classes


In [2]:
#X = torch.load('/home/evan/D1/project/code/raw_cls_tokens_features.pt', map_location=device)
#X = torch.load('/home/evan/D1/project/code/stretched_cls_tokens.pt', map_location=device)
#X = torch.load('/home/evan/D1/project/code/reflected_cls_tokens.pt', map_location=device)

y = np.load('/home/evan/D1/project/code/cls_tokens_labels.npy')
frame_counts = np.load('/home/evan/D1/project/code/frame_counts.npy')


class_mapping = {0:0, 1: 1, 2: 2, 3: 1, 4: 2}

for i, label in enumerate(y):
    y[i] = class_mapping[label]
print('Done')

## Helper functions to split sequence, extract and (add context frames - inactive)

In [3]:
def split_sequences_np(arr):
    """Find the difference between consecutive elements"""
    
    diffs = np.diff(arr)
    # Identify where the difference is not 1 (i.e., breaks in consecutive sequences)
    breaks = np.where(diffs != 1)[0] + 1
    
    # Use numpy split to divide the array at every break point
    return np.split(arr, breaks)

def extract_data(games, boundaries, X, y, indices):
    X_split, y_split, idx_split = [], [], []
    for g in games:
        start = 0 if g == 0 else boundaries[g-1]
        end = boundaries[g]
        X_split.append(X[start:end])
        y_split.append(y[start:end])
        idx_split.extend(indices[start:end])
    return torch.cat(X_split), torch.cat(y_split), idx_split

def add_context_frames(seq, total_frames, context_size=0, last_context_end=0):
    """Adds context frames to the sequence, ensuring no overlap with other sequences or video boundaries"""
    seq_start_idx, seq_end_idx = seq[0], seq[-1]
    
    # Calculate start and end of context
    start_with_context = max(seq_start_idx - context_size, last_context_end + 1, 0)
    end_with_context = min(seq_end_idx + context_size, total_frames - 1)
    
    return start_with_context, end_with_context


## Undersample tackle-sequences

In [ ]:
import numpy as np
import torch

def extract_tackle_sequences_and_undersample(X, y, frame_counts, device='cuda', max_tackles=500):
    new_X, new_y, kept_indices = [], [], []
    all_seq = []
    class_lengths = {}  # Stores sequence lengths and counts by class
    total_sequences = 0  # Total count of all sequences
    tackle_count = 0  # Counter for class 0 sequences
    tackle_count2 = 0  # Counter for class 2 sequences

    start_idx = 0
    last_context_end = -1

    for z, count in enumerate(frame_counts):
        end_idx = start_idx + count

        key_frame_indices = np.where(y[start_idx:end_idx] != 0)[0] + start_idx
        seq_splitted = split_sequences_np(key_frame_indices)

        background_frame_indices = np.where(y[start_idx:end_idx] == 0)[0] + start_idx
        bg_seq_splitted = split_sequences_np(background_frame_indices)

        for bg_seq in bg_seq_splitted:
            if len(bg_seq) >= 70:
                if y[bg_seq[0]] == 0:  # Check if the sequence is of class 0
                    if tackle_count >= max_tackles:
                        continue  # Skip if maximum number of tackles has been reached
                    else:
                        tackle_count += 1
                start_random = np.random.randint(35, len(bg_seq) - 34)
                random_seq = bg_seq[start_random:start_random + 25]
                all_seq.append(random_seq)
                total_sequences += 1

                class_id = y[random_seq[0]]
                if class_id not in class_lengths:
                    class_lengths[class_id] = {'lengths': [], 'count': 0}
                class_lengths[class_id]['lengths'].append(len(random_seq))
                class_lengths[class_id]['count'] += 1

                new_bg_seq_x = X[random_seq[0]:random_seq[-1] + 1]
                new_bg_seq_y = y[random_seq[0]:random_seq[-1] + 1]

                new_X.extend(new_bg_seq_x)
                new_y.extend(new_bg_seq_y)

        for seq in seq_splitted:
            if seq.size > 0:
                if y[seq[0]] == 2:  # Check if the sequence is of class 0
                    if tackle_count2 >= 280:
                        continue  # Skip if maximum number of tackles has been reached
                    else:
                        tackle_count2 += 1

                all_seq.append(seq)
                total_sequences += 1

                class_id = y[seq[0]]
                if class_id not in class_lengths:
                    class_lengths[class_id] = {'lengths': [], 'count': 0}
                class_lengths[class_id]['lengths'].append(len(seq))
                class_lengths[class_id]['count'] += 1

                
                # This line will return the same values for start and end, as 0 context is added. background is sampled randomly instead in loop before this.
                context_start, context_end = add_context_frames(seq, end_idx, last_context_end=last_context_end)
                
                new_X.extend(X[context_start:context_end + 1])
                new_y.extend(y[context_start:context_end + 1])

                last_context_end = context_end

        start_idx = end_idx

    new_X = torch.stack(new_X)
    new_X = torch.tensor(new_X, dtype=torch.float32, device=device)
    new_y = torch.tensor(new_y, dtype=torch.long, device=device)

    return new_X, new_y, kept_indices, all_seq, class_lengths, total_sequences

new_X, new_y, kept_indices, all_seq, class_lengths, total_sequences = extract_tackle_sequences_and_undersample(X, y, frame_counts)

# Compute and print average lengths and counts for each class
for class_id, info in class_lengths.items():
    average_length = np.mean(info['lengths'])
    print(f'Class {class_id} - Average Sequence Length: {average_length:.2f}, Count: {info["count"]}')

print(f'Total sequences processed: {total_sequences}')
print(new_X.shape)
print(new_y.shape)
print(np.unique(new_y.cpu(), return_counts=True))


## Split games into either train, val, test to ensure no data leakage

In [7]:
import numpy as np
import torch

def split_data_by_game(X, y, kept_indices, frame_counts, split_ratio=(0.7, 0.15, 0.15), seed=None):
    # Set the random seed for reproducibility
    np.random.seed(seed)
    
    # Calculate the cumulative sum of frame counts to determine game boundaries
    boundaries = np.cumsum(frame_counts)
    
    # Create lists to hold data for train, validation, and test sets
    X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []
    idx_train, idx_val, idx_test = [], [], []
    
    # Shuffle the indices to randomize which games go into which set
    game_indices = np.arange(len(frame_counts))
    np.random.shuffle(game_indices)
    
    total_games = len(game_indices)
    num_train = int(total_games * split_ratio[0])
    num_val = int(total_games * split_ratio[1])
    
    print('Number of total games: ', total_games)
    print('Number of train games: ', num_train)
    print('Number of val games: ', num_val)
    
    
    # Assign games to train, validation, and test sets
    train_games = game_indices[:num_train]
    val_games = game_indices[num_train:num_train + num_val]
    test_games = game_indices[num_train + num_val:]


    # Extract data for each split
    X_train, y_train, idx_train = extract_data(train_games, boundaries, X, y, kept_indices)
    X_val, y_val, idx_val = extract_data(val_games, boundaries, X, y, kept_indices)
    X_test, y_test, idx_test = extract_data(test_games, boundaries, X, y, kept_indices)
    
    return (X_train, y_train, idx_train), (X_val, y_val, idx_val), (X_test, y_test, idx_test)

# Set the random seed for reproducibility
seed = 42

split_ratio = (0.7, 0.15, 0.15)
(X_train, y_train, idx_train), (X_val, y_val, idx_val), (X_test, y_test, idx_test) = split_data_by_game(new_X, new_y, kept_indices, frame_counts, split_ratio, seed)

print(np.unique(y_train.cpu(), return_counts=True)[1])
print(np.unique(y_val.cpu(), return_counts=True)[1])
print(np.unique(y_test.cpu(), return_counts=True)[1])


## Create tensors

In [8]:
X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train, dtype=torch.long, device=device)

X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
y_val = torch.tensor(y_val, dtype=torch.long, device=device)

X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test = torch.tensor(y_test, dtype=torch.long, device=device)

## Shifting Window - Dataset + DataLoaders 

In [143]:
class FrameSequenceDataset(TensorDataset):
    def __init__(self, X, y, window_size):
        self.X = X
        self.y = y
        self.window_size = window_size
    
    def __len__(self):
        # Calculate how many complete non-overlapping windows fit into the dataset
        return (len(self.y) // self.window_size)
    
    def __getitem__(self, index):
        # Calculate the start and end of the sequence based on the window size and index
        start = index * self.window_size
        end = start + self.window_size
        X_seq = self.X[start:end]
        
        y_seq = self.y[start:end]
        return X_seq, y_seq

window_size = 75  # 25 fps
train_dataset = FrameSequenceDataset(X_train, y_train, window_size)
val_dataset = FrameSequenceDataset(X_val, y_val, window_size)
test_dataset = FrameSequenceDataset(X_test, y_test, window_size)


train_loader = DataLoader(train_dataset, batch_size=256, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

## Sliding Window - Dataset + DataLoaders 

In [152]:
class FrameSequenceDataset(TensorDataset):
    def __init__(self, X, y, window_size):
        self.X = X
        self.y = y
        self.window_size = window_size
    
    def __len__(self):
        # Adjust the length to allow for complete windows only
        return len(self.y) - self.window_size + 1
    
    def __getitem__(self, index):
        # Extract a window of data starting at `index`
        X_seq = self.X[index:index + self.window_size]
        y_seq = self.y[index:index + self.window_size]
        return X_seq, y_seq


window_size = 25  # 25 fps
train_dataset = FrameSequenceDataset(X_train, y_train, window_size)
val_dataset = FrameSequenceDataset(X_val, y_val, window_size)
test_dataset = FrameSequenceDataset(X_test, y_test, window_size)


train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

## TempTAC and Attention-module

In [35]:
class AttentionModule(nn.Module):
    def __init__(self, input_size, heads):
        super().__init__()
        self.self_attention = nn.MultiheadAttention(embed_dim=input_size, num_heads=heads)
    
    def forward(self, x):
        # x: [seq_len, batch_size, features]
        x = x.permute(1, 0, 2)  # Adjusting for MultiheadAttention
        attn_output, _ = self.self_attention(x, x, x)
        return attn_output.permute(1, 0, 2)  # Return same shape



class TempTAC(nn.Module):
    def __init__(self, input_size, hidden_size, output_dim, num_layers, device, dropout_prob=0.5):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size + output_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout_prob, bidirectional=True)
        self.linear = nn.Linear(hidden_size * 2, output_dim)
        self.dropout = nn.Dropout(dropout_prob)
        self.device = device
        self.attention = AttentionModule(input_size, heads=8)  # Attention layer after LSTM
        self.prev_output_weight = nn.Parameter(torch.tensor(1.0))  # Initialize learnable weight for previous output influence
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_dim = output_dim
        
        
    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(self.device),
                torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(self.device))

    def forward(self, x, hidden, current_batch_size):
        batch_size, seq_len, _ = x.size()
        prev_output = torch.zeros(batch_size, 1, self.output_dim, device=self.device)
        
        # Apply attention to the output of the current timestep
        x = self.attention(x)  # Remove sequence length dimension for attention
        
        outputs = []
        for t in range(seq_len):
            weighted_prev_output = prev_output * self.prev_output_weight  # Apply learned weight to previous output
            
            lstm_input = torch.cat((x[:, t:t+1, :], weighted_prev_output), dim=-1)
            
            lstm_out, hidden = self.lstm(lstm_input, hidden)
            lstm_out = self.dropout(lstm_out)
            
            prev_output = self.linear(lstm_out)  # Linear layer and unsqueeze to match dimensions
            outputs.append(prev_output)

        return torch.cat(outputs, dim=1), hidden

### Models parameter counter

In [317]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#model = EnhancedMultiLayerClassifier(1024, 3)
print("Number of trainable parameters:", count_parameters(model))
print("Number of training instances:", sum(np.unique(y_train.cpu(), return_counts=True)[1]))

## Training Loop

In [133]:
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau


config = {'l1_lambda': 3.326004484093452e-05,
'lr': 0.0004256783934105,
'weight_decay': 1.4334181994254526e-05}


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    training_losses = []
    validation_losses = []
    
    best_val = 100
    break_margin = 2
    
    best_f1 = 0.0

    l1_lambda = config['l1_lambda']

    batch_size = 256
    current_lr = optimizer.param_groups[0]['lr']
    
    
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
                
        # Initialize hidden state for the first batch size
        initial_batch = next(iter(train_loader))
        
        initial_batch_size = initial_batch[0].size(0)
        
        hidden = model.init_hidden(initial_batch_size)
        clip_value = 1
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            current_batch_size = inputs.size(0)
            
            # Adjust hidden state size if current batch size differs from the initial batch size
            if current_batch_size != initial_batch_size:
                adjusted_hidden = (hidden[0][:, :current_batch_size, :].contiguous(),
                                   hidden[1][:, :current_batch_size, :].contiguous())

            else:
                adjusted_hidden = hidden
            
            outputs, adjusted_hidden = model(inputs, adjusted_hidden, current_batch_size)
            
            hidden = tuple([h.data for h in adjusted_hidden])  # Detach hidden state from graph to prevent backprop through entire dataset
            
            optimizer.zero_grad()
            
            
            # Calculate loss for the entire sequence at once
            loss = criterion(outputs.transpose(1, 2), labels)
            
            #l1_norm = sum(torch.linalg.norm(p, 1) for p in model.parameters())
            l1_norm = sum(p.abs().sum() for p in model.parameters())

            loss = loss + l1_lambda * l1_norm
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            total_train_loss += loss.item()

        average_train_loss = total_train_loss / len(train_loader)
        training_losses.append(average_train_loss) 
        
        
        model.eval()
        total_val_loss = 0
        all_preds = []
        all_targets = []
        total, correct = 0, 0
        with torch.no_grad():
            initial_batch = next(iter(val_loader))
            initial_batch_size = initial_batch[0].size(0)
            hidden = model.init_hidden(initial_batch_size)
            
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                current_batch_size = inputs.size(0)
                
                # Adjust hidden state size if current batch size differs from the initial batch size
                if current_batch_size != initial_batch_size:
                    adjusted_hidden = (hidden[0][:, :current_batch_size, :].contiguous(),
                                       hidden[1][:, :current_batch_size, :].contiguous())
                else:
                    adjusted_hidden = hidden
    
                outputs, adjusted_hidden = model(inputs, adjusted_hidden, current_batch_size)
                val_loss = criterion(outputs.transpose(1, 2), labels)
                total_val_loss += val_loss.item()

                for i in range(outputs.shape[1]):
                    _, predicted = torch.max(outputs[:, i, :].data, 1)
                    total += labels[:, i].size(0)
                    correct += (predicted.cpu() == labels[:, i].cpu()).sum().item()
                    
                    all_preds.extend(predicted.cpu().numpy())
                    all_targets.extend(labels[:, i].cpu().numpy())

        average_val_loss = total_val_loss / len(val_loader)
        validation_losses.append(average_val_loss) 
        accuracy = 100 * correct / total
        scheduler.step(val_loss)
        
        precision, recall, f1, _ = precision_recall_fscore_support(np.array(all_targets).flatten(), np.array(all_preds).flatten(), average='weighted', zero_division=0)
        
        if f1 > best_f1:
            best_f1 = f1
            best_epoch = epoch
            best_model_state_dict = model.state_dict()
            best_all_targets = all_targets
            best_all_preds = all_preds
        
        if (int(epoch) % 5) == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_train_loss:.4f}, Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.2f}%')
            print(f'Epoch [{epoch+1}/{num_epochs}] Current Learning Rate: {current_lr}')
            
        
        current_lr = optimizer.param_groups[0]['lr']
        
        if average_val_loss < best_val:
            best_val = average_val_loss
            time_to_break = 0
            print('New best val loss: ', average_val_loss)
        else:
            time_to_break += 1
        
        if time_to_break > break_margin:
            print('Break margin hit!')
            break

        
    return best_model_state_dict, best_all_targets, best_all_preds, validation_losses, training_losses


model = TempTAC(input_size=1024, hidden_size=256, output_dim=3, num_layers=2, device=device, dropout_prob=0.5)
model

classes = np.unique(y_train.cpu())
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train.cpu().numpy())
class_weights = torch.tensor(weights, dtype=torch.float32).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])

criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4, verbose=True)

best_model_state_dict, all_targets, all_preds, validation_losses, training_losses = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=400)

all_targets = np.array(all_targets).flatten()
all_preds = np.array(all_preds).flatten()

model.load_state_dict(best_model_state_dict)

print(classification_report(all_targets, all_preds, target_names=['background', 'tackle-live', 'tackle-replay']))


## Optimize hyperparams

In [ ]:
import logging
import sys
import torch
import pandas as pd
import torch.nn as nn
from sklearn.metrics import classification_report
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau


def objective(trial, train_loader, val_loader, num_epochs=10, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    
    model = TempTAC(input_size=1024, hidden_size=256, output_dim=3, num_layers=2, device=device, dropout_prob=0.5)

    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    training_losses, validation_losses = [], []
    
    best_val = float('inf')
    break_margin = 3
    best_f1 = 0.0
    
    # Updated to use suggest_float with log=True
    lr = trial.suggest_float('lr', 1e-6, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
    l1_lambda = trial.suggest_float('l1_lambda', 0, 0.001)
    
    classes = np.unique(y_train.cpu())
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train.cpu().numpy())
    class_weights = torch.tensor(weights, dtype=torch.float32).to(device)


    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss(weight=class_weights).to(device) 
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4, verbose=True)
    
    batch_size = 256
    
    current_lr = optimizer.param_groups[0]['lr']
    
    
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        # Initialize hidden state for the first batch size
        initial_batch = next(iter(train_loader))
        
        initial_batch_size = initial_batch[0].size(0)
        
        hidden = model.init_hidden(initial_batch_size)
        clip_value = 1
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            current_batch_size = inputs.size(0)
            
            # Adjust hidden state size if current batch size differs from the initial batch size
            if current_batch_size != initial_batch_size:
                adjusted_hidden = (hidden[0][:, :current_batch_size, :].contiguous(),
                                   hidden[1][:, :current_batch_size, :].contiguous())

            else:
                adjusted_hidden = hidden
            
            outputs, adjusted_hidden = model(inputs, adjusted_hidden, current_batch_size)
            
            hidden = tuple([h.data for h in adjusted_hidden])  # Detach hidden state from graph to prevent backprop through entire dataset
            
            optimizer.zero_grad()
            
            # Calculate loss for the entire sequence at once
            loss = criterion(outputs.transpose(1, 2), labels)
            
            #l1_norm = sum(torch.linalg.norm(p, 1) for p in model.parameters())
            l1_norm = sum(p.abs().sum() for p in model.parameters())

            loss = loss + l1_lambda * l1_norm
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value) # gradient clipping
            optimizer.step()
            total_train_loss += loss.item()

        average_train_loss = total_train_loss / len(train_loader)
        training_losses.append(average_train_loss) 
        
        
        model.eval()
        total_val_loss = 0
        all_preds = []
        all_targets = []
        total, correct = 0, 0
        with torch.no_grad():
            initial_batch = next(iter(val_loader))
            initial_batch_size = initial_batch[0].size(0)
            hidden = model.init_hidden(initial_batch_size)
            
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                current_batch_size = inputs.size(0)
                
                # Adjust hidden state size if current batch size differs from the initial batch size
                if current_batch_size != initial_batch_size:
                    adjusted_hidden = (hidden[0][:, :current_batch_size, :].contiguous(),
                                       hidden[1][:, :current_batch_size, :].contiguous())
                else:
                    adjusted_hidden = hidden
    
                outputs, adjusted_hidden = model(inputs, adjusted_hidden, current_batch_size)
                val_loss = criterion(outputs.transpose(1, 2), labels)
                total_val_loss += val_loss.item()

                for i in range(outputs.shape[1]):
                    _, predicted = torch.max(outputs[:, i, :].data, 1)
                    total += labels[:, i].size(0)
                    correct += (predicted.cpu() == labels[:, i].cpu()).sum().item()
                    
                    all_preds.extend(predicted.cpu().numpy())
                    all_targets.extend(labels[:, i].cpu().numpy())

        average_val_loss = total_val_loss / len(val_loader)
        validation_losses.append(average_val_loss) 
        accuracy = 100 * correct / total
        scheduler.step(average_val_loss)
        
        precision, recall, f1, _ = precision_recall_fscore_support(np.array(all_targets).flatten(), np.array(all_preds).flatten(), average='weighted', zero_division=0)
        
        if f1 > best_f1:
            best_f1 = f1
            best_epoch = epoch
            best_model_state_dict = model.state_dict()
            best_all_targets = all_targets
            best_all_preds = all_preds
        
        current_lr = optimizer.param_groups[0]['lr']
        
        if val_loss < best_val:
            best_val = val_loss
            time_to_break = 0
        else:
            time_to_break += 1
        
        if time_to_break > break_margin:
            #print('Break margin hit!')
            break
            
        trial.report(f1, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
            
        

        
    return f1


study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())

optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))

study.optimize(lambda trial: objective(trial, train_loader, val_loader, num_epochs=1000, device=device), n_trials=100)


# Save study data to a df
study_data = study.trials_dataframe()

# Save the df to a csv
study_data.to_csv('study_data.csv', index=False)

print("Best trial:")
trial = study.best_trial
print(f" Value: {trial.value}")
print(" Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

## Read saved optuna-study

In [27]:
# Load the csv saved above
study_data = pd.read_csv('study_data.csv')

# Find the best trial (higher value is better = idxmax, min use idxmin)
best_trial = study_data.loc[study_data['value'].idxmax()]

# Print info about best trial
print("Best Trial:")
print(best_trial)

# Print parameters of the best trial
print("\nBest Trial Parameters:")
for param in [col for col in study_data.columns if col.startswith('params_')]:
    print(f"{param.replace('params_', '')}: {best_trial[param]}")


In [1250]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
conf_matrix = confusion_matrix(np.array(all_targets).flatten(), np.array(all_preds).flatten())
# conf_matrix = confusion_matrix(all_preds, all_targets)
labels = ["background", "tackle-live", "tackle-replay"]
#labels = ["background", "tackle-live", "tackle-replay", "tackle-live-incomplete", "tackle-replay-incomplete"]

 
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
# plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("new_best_8_context_frames_window_25.pdf", format="pdf", bbox_inches="tight")
plt.show()

## Plot loss

In [134]:
scaler = MinMaxScaler()

length=len(training_losses)

# For better plotting, we normalize
training_losses_normalized = scaler.fit_transform(np.array(training_losses).reshape(-1, 1)).flatten()
validation_losses_normalized = scaler.fit_transform(np.array(validation_losses).reshape(-1, 1)).flatten()

# Update df with normalized values
df = pd.DataFrame({
    'Epoch': np.arange(length),
    'Train Loss': training_losses_normalized,
    'Validation Loss': validation_losses_normalized
})

# Plot normalized losses
plt.figure(figsize=(10, 6))
plt.plot(df['Epoch'], df['Train Loss'], label='Train Loss (Normalized)', marker='o')
plt.plot(df['Epoch'], df['Validation Loss'], label='Validation Loss (Normalized)', marker='o')
plt.title('Training vs Validation Loss (Normalized)')
plt.xlabel('Epoch')
plt.ylabel('Normalized Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save loss
#np.save('Optimized-reflected-run/training_losses-undersampled-window-75-sliding-window.npy', training_losses)
#np.save('Optimized-reflected-run/validation_losses-undersampled-window-75-sliding-window.npy', validation_losses)
#plt.savefig(f'Optimized-reflected-run/train_val_loss-undersampled-window-75-sliding-window.pdf', format='pdf')

plt.show()

## Run inference on test-set

In [153]:
# For loading prev models
# model.load_state_dict(torch.load('Optimized-reflected-run/sliding-window/best_lstm-undersampled-window-25-sliding-window.pt'))

def run_inference_test_set():   
    all_targets = []
    all_preds = []

    initial_batch_size = 256

    initial_batch = next(iter(test_loader))

    initial_batch_size = initial_batch[0].size(0)

    hidden = model.init_hidden(initial_batch_size)

    total_val_loss = 0
    total = 0
    correct = 0

    for i, (inputs, labels) in enumerate(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        current_batch_size = inputs.size(0)

        # Adjust hidden state size if current batch size differs from the initial batch size
        if current_batch_size != initial_batch_size:
            adjusted_hidden = (hidden[0][:, :current_batch_size, :].contiguous(),
                               hidden[1][:, :current_batch_size, :].contiguous())
        else:
            adjusted_hidden = hidden



        outputs, adjusted_hidden = model(inputs, adjusted_hidden, current_batch_size)

        val_loss = criterion(outputs.transpose(1, 2), labels)
        total_val_loss += val_loss.item()

        for i in range(outputs.shape[1]):
            _, predicted = torch.max(outputs[:, i, :].data, 1)
            total += labels[:, i].size(0)
            correct += (predicted.cpu() == labels[:, i].cpu()).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels[:, i].cpu().numpy())
    return all_preds, all_targets   


test_predictions, test_targets  = run_inference_test_set()


# Plot classification report
print(classification_report(test_targets, test_predictions, target_names=['background', 'tackle-live', 'tackle-replay']))

# Create CM
conf_matrix = confusion_matrix(test_targets, test_predictions)


labels = ["background", "tackle-live", "tackle-replay"]

 
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
# plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("Optimized-reflected-run/best_lstm-undersampled-window-75-sliding-window.pdf", format="pdf", bbox_inches="tight") 
plt.show()

#torch.save(model.state_dict(), 'Optimized-reflected-run/best_lstm-undersampled-window-75-sliding-window.pt')

## Print ROC-curves

In [154]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

class_names = ['background', 'tackle-live', 'tackle-replay']
n_classes = len(class_names)

# binarize the targets and predictions for roc curve computation
test_targets_bin = label_binarize(test_targets, classes=[0, 1, 2])
test_predictions_bin = label_binarize(test_predictions, classes=[0, 1, 2])

# roc curve and auc for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(test_targets_bin[:, i], test_predictions_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# plot rpc curves for each class
plt.figure(figsize=(8, 6))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')
    
    
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.grid(visible=True)
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curve')
plt.legend(loc='lower right')
#plt.savefig("Optimized-reflected-run/roc-curve-25-sliding-window.pdf", format="pdf", bbox_inches="tight")
plt.show()

## Baseline class

In [166]:
class Simple1DCNN(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(Simple1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * (sequence_length // 2), num_classes)  # adjust based on pooling and input length

    def forward(self, x):
        # x.shape = (batch, channels, sequence_length)
        x = F.relu(self.conv1(x))
        x = F.max_pool1d(x, kernel_size=2)
        
        # flatten to fully connected layer
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        return x

## Loss plotting

In [245]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

epochs = len(training_losses)  # number of epochs

df = pd.DataFrame({
    'Epoch': [i for i in range(epochs)],
    'Train Loss': training_losses,
    'Validation Loss': validation_losses
})

plt.figure(figsize=(10, 6))
plt.plot(df['Epoch'], df['Train Loss'], label='Train Loss', marker='o')
plt.plot(df['Epoch'], df['Validation Loss'], label='Validation Loss', marker='o')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.savefig(f'baseline_cnn_training_validation_loss_plot.pdf', format='pdf')
plt.show()